In [7]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from keras.models import Sequential 
from keras.datasets import mnist
import keras



from sklearn.model_selection import train_test_split

In [8]:
EPOCHS=10
IMG_WIDTH=30
IMG_HEIGHT=30
NUM_CATEGORIES=43
TEST_SIZE=0.4

In [174]:
def load_data(data_dir="C:\\Users\\jasmin olabi\\.conda\\envs\\tf\\Lib\\site-packages\\traffic\\gtsrb"):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images=[]
    labels=[]
    
    for root, _, files in os.walk(data_dir):
        for file in files:
            if not file.startswith('.'):
                # Read in and resize image
                img = cv2.imread(os.path.join(root, file))
                img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

                # Add current image and label to output lists
                images.append(img)
                labels.append(int(os.path.basename(root)))

    return (images, labels)


In [175]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    # Create a convolutional neural network
    model = tf.keras.models.Sequential([

        # Convolutional layer. Learn 32 filters using a 3x3 kernel
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),

        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # 2nd convolutional layer. Learn 32 filters using a 3x3 kernel
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),

        # Flatten units
        tf.keras.layers.Flatten(),

        # Add a hidden layer with dropout
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),

        # Add an output layer with output unit for all categories
        tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
    ])

    # Train neural network
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [176]:
def main():

    # Check command-line arguments
    if len(sys.argv) not in [1, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    images, labels = load_data("C:\\Users\\jasmin olabi\\.conda\\envs\\tf\\Lib\\site-packages\\traffic\\gtsrb")

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels,NUM_CATEGORIES)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )

    # Get a compiled neural network
    model = get_model()
    
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)
    
    print(len(sys.argv))

    # Save model to file
    if len(sys.argv) == 1:
        filename = 'C:\\Users\\jasmin olabi\\.conda\\envs\\tf\\Lib\\site-packages'
        model.save("model",saved_format="h5")
        print(f"Model Saved to {filename},")

In [177]:
if __name__ == "__main__":
    main()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 43)               